Goals of this analysis: 
- Create **rolling mean** of data to smooth out some variation
- Make separate **linear regressions** for distinct treatment periods

In [3]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns

In [29]:
data=pd.read_excel('PerformanceDataFake.xlsx')
data.head(n=15)

,Subject,Measure1,Measure2,Measure3,Measure4,Measure5,TestPhase,SessionLabel2,SessionLabel,LastTreatmentSession,InTreatmentSessionNumber
0,1,95,14,3,93,39,-1,P1,P6,0,NaN
1,1,28,15,51,54,28,-1,P2,P5,0,NaN
2,1,70,77,13,35,63,-1,P3,P4,0,NaN
3,1,5,65,45,62,50,-1,P4,P3,0,NaN
4,1,38,62,76,53,63,-1,P5,P2,0,NaN
5,1,25,85,29,4,75,-1,P6,P1,0,NaN
6,1,56,23,20,64,43,0,F1,F4,0,1.0
7,1,84,7,49,26,64,0,F2,F3,0,2.0
8,1,33,56,37,51,81,0,F3,F2,0,3.0
9,1,70,59,33,27,72,0,F4,F1,1,4.0


Each row is one trial for one subject. Several measures are taken, and metadata is included to denote whether this trial occured before treatment began, during treatment, or after treatment ended.

In [30]:
#Sanitize the inputs a little
df = data

#for this analysis, we don't need post-treatment -- remove it
df = df[df['SessionLabel2'].str.contains('R1')==False] 

#For our linear regression, we need NaNs replaced with 0
df['In-TreatmentSessionNumber'].fillna(0,inplace=True)

#And change TestPhase from -1,0 to 0,1
df = df.assign(TestPhase = [0 if TestPhase == -1 else 1 for TestPhase in df['TestPhase']]) #Convert the -1 to 0

#We want a test session number for each session by subject
df.reset_index() #make sure index corresponds to row
sessNum=[]
prevSub=0
for ind,row in df.iterrows():
    #if prevSub == df.loc[ind,'Subject']:
    if prevSub == df['Subject'][ind]:
        count+=1
    else:
        count=1
    sessNum.append(count)
    prevSub=df['Subject'][ind]

#assign
df = df.assign(TestSessionNo = sessNum)



#Variables to loop through
subjects=df['Subject'].unique()
measures=['Measure1','Measure2','Measure3','Measure4','Measure5']

KeyError: 'In-TreatmentSessionNumber'

The above warning is a false positive **(** and can be disabled with pd.set_option('mode.chained_assignment',None) **)**

We need to make two functions now -- one to implement a rolling mean, and one to plot our data and fit lines

In [24]:
def rolling_plot_AT(sub,measure,noplot=False): 

    
    # get a rolling mean (each point is avgd with point before it)
    rmean = sub[measure].rolling(window=2).mean() #getting rolling mean
    
    #But we have two problems -- we want to keep our first time point (it will remain unchanged), and
    #   we don't want to average timepoints from different treatment phases together.
    
    #HERE, I replace first point with first point (instead of NAN)
    rmean.loc[sub['SessionLabel2'] == 'P1']=sub.loc[sub['SessionLabel2'] == 'P1',measure] 
    
    # replace first in-treatment point that is avgd with pre-treatment with first in-treatment (not averaged)
    rmean.loc[sub['SessionLabel2'] == 'F1']=sub.loc[sub['SessionLabel2'] == 'F1',measure]
    
    # plot rolling mean
    if not noplot:
        plt.figure(figsize=(10,6))
        orig = plt.plot(sub[measure], color='black',label='Original', marker='.')
        mean = plt.plot(rmean, color='red',label='Rolling Mean', marker='.')
        plt.legend(loc='best')
        plt.title("Rolling mean")
        plt.show(block=False)
    
    return rmean

In [25]:
# Get model predictions and 95% confidence interval
def counterfactual_plot_AT(res, df, measure1): 
    start = df.loc[df['TestPhase']==1].index[0]
    end = df.loc[df['TestPhase']==1].index[-1]
    beta = res.params
    predictions = res.get_prediction(df) 
    summary = predictions.summary_frame(alpha=0.05)

    # mean predictions
    y_pred = predictions.predicted_mean

    # countefactual assumes no interventions
    cf_df = df.copy()
    cf_df["TestPhase"] = 0.0
    cf_df["In-TreatmentSessionNumber"] = 0.0

    # counter-factual predictions
    cf = res.get_prediction(cf_df).summary_frame(alpha=0.05)

    # Plotting
    plt.style.use('seaborn-whitegrid')
    fig, ax = plt.subplots(figsize=(16,10))

    # Plot bounce rate data
    ax.scatter(df["TestSessionNo"], df['rmean_'+measure1], facecolors='none', edgecolors='steelblue', label='Moving Avg '+measure1+' Score', linewidths=2)

    # Plot model mean bounce rate prediction
    ax.plot(df["TestSessionNo"][:start], y_pred[:start], 'b-', label="model prediction")
    ax.plot(df["TestSessionNo"][start:], y_pred[start:], 'b-')

    # Plot counterfactual mean bounce rate with 95% confidence interval
    ax.plot(df["TestSessionNo"][start:], cf['mean'][start:], 'k.', label="counterfactual")
    ax.fill_between(df["TestSessionNo"][start:], cf['mean_ci_lower'][start:], cf['mean_ci_upper'][start:], color='k', alpha=0.1, label="counterfactual 95% CI");

    # Plot line marking intervention moment
    ax.axvline(x = start+1, color = 'r', label = 'intervention')

    ax.legend(loc='best')
    #plt.ylim([10, 15])
    plt.xlabel("Test Session")
    plt.ylabel(measure1)
    plt.show()
    
    

Now it's time to create fit lines for our treatment conditions and plot these for each subject/measure

In [28]:
#A little setup to allow us to do nested assignment
from collections import defaultdict
model = defaultdict(dict)
res = defaultdict(dict)

#import models

import statsmodels.formula.api as smf

# time for big loops!
for subject in subjects:
     print(df[df['Subject']==subject])
     subdf = df[df['Subject']==subject]
     subdf = subdf[subdf['SessionLabel2'].str.contains("P|F")==True] #get InTreatment and pre-
     subdf = subdf[['Subject','TestPhase','TestSessionNo','SessionLabel2','InTreatmentSessionNumber',*measures]]
     subdf = subdf.set_index(['TestSessionNo'],drop=False)
    
     for measure in measures:
        print('using measure '+measure)
         #rolling_plot
        subdf['rmean_'+measure]=rolling_plot_AT(subdf,measure,noplot=True)
        subdf.reset_index(drop=True,inplace=True)
    
        ##Ordinary least squares for effect of "Test Phase"
        model[subject][measure] = smf.ols(formula='rmean_'+measure+' ~ TestSessionNo + TestPhase + In\-TreatmentSessionNumber', data=subdf)
        res[subject][measure] = model[subject][measure].fit()
        print(res[subject][measure].summary())

        #Plot
        counterfactual_plot_AT(res[subject][measure],subdf,measure)

   Subject  Measure1  Measure2  Measure3  Measure4  Measure5  TestPhase  \
0        1        95        14         3        93        39          0   
1        1        28        15        51        54        28          0   
2        1        70        77        13        35        63          0   
3        1         5        65        45        62        50          0   
4        1        38        62        76        53        63          0   
5        1        25        85        29         4        75          0   
6        1        56        23        20        64        43          1   
7        1        84         7        49        26        64          1   
8        1        33        56        37        51        81          1   
9        1        70        59        33        27        72          1   

  SessionLabel2 SessionLabel  LastTreatmentSession  In-TreatmentSessionNumber  \
0            P1           P6                     0                        0.0   
1           

PatsyError: error tokenizing input (maybe an unclosed string?)
    rmean_Measure1 ~ TestSessionNo + TestPhase + In\-TreatmentSessionNumber
                                                   ^